In [ ]:
from utils import *

In [23]:
data_folder = 'data'

#### Query Lists

In [18]:
#Query tree
list_1= ["large language model","LLM"]
list_3= ["medicine","healthcare","cancer"]
list_2 = [
"Retrieved augmented generation",
"Knowledge graph",
"Graph database",
"Knowledge base",
"Agents",
"agentic",
"chatgpt",
"llama",
"ULM"]
list_4 =[
"patient care",
"patient monitoring",
"imaging",
"decision support",
"diagnosis",
"treatment",
"question answering",
"hallucination"]

LomgFormat: This list the whole query, as [main_domain] AND [tools] AND [medical_designation] AND [medical_domain]



In [19]:
combined_list = []
for el1 in list_1:
    for el2 in list_2:
        for el3 in list_3:
            for el4 in list_4:
                combined_element = f'"{el1}" AND "{el2}" AND "{el3}" AND "{el4}"'
                combined_list.append(combined_element)

ShortFormat: This query list skips tool , as [main_domain]  AND [medical_designation] AND [medical_domain]

In [ ]:
combined_list = []
for el1 in list_1:
        for el3 in list_3:
            for el4 in list_4:
                combined_element = f'"{el1}" AND "{el3}" AND "{el4}"'
                combined_list.append(combined_element)

In [10]:
#sample
combined_list[:5]

['"large language model" AND "medicine" AND "patient care"',
 '"large language model" AND "medicine" AND "patient monitoring"',
 '"large language model" AND "medicine" AND "imaging"',
 '"large language model" AND "medicine" AND "decision support"',
 '"large language model" AND "medicine" AND "diagnosis"']

#### Arxiv Sweep

In [ ]:
year_start=2024
filename = 'queries_arxiv_v2_1.xlsx'


if year_start or year_end:
        current_year = datetime.now().year
        year_start = year_start 
        year_end =  current_year
        date_range = (year_start, year_end)
df_arxiv=pd.DataFrame()
for query in combined_list:
  df_temp =search_and_export(query, 1000, year_start=None, year_end=None, filename=None)# search_arxiv(query, max_results=2000, start=0, sort_by='relevance', date_range=None)
  try:
    print("Query:"+query+str(df_temp.shape))
  except:
    print("Query:"+query+str(None))
  df_arxiv=pd.concat([df_arxiv,df_temp])
  df_arxiv.to_excel(os.path.join(data_folder, filename))
  time.sleep(10)

Consultando arXiv: "large language model" AND "medicine" AND "patient care"
Obtenidos 10 resultados en este lote (total: 10)
Consultando arXiv: "large language model" AND "medicine" AND "patient care"
Totalresultados: 10
Query:"large language model" AND "medicine" AND "patient care"(10, 12)
Consultando arXiv: "large language model" AND "medicine" AND "patient monitoring"
Obtenidos 1 resultados en este lote (total: 1)
Consultando arXiv: "large language model" AND "medicine" AND "patient monitoring"
Totalresultados: 1
Query:"large language model" AND "medicine" AND "patient monitoring"(1, 12)
Consultando arXiv: "large language model" AND "medicine" AND "imaging"
Obtenidos 25 resultados en este lote (total: 25)
Consultando arXiv: "large language model" AND "medicine" AND "imaging"
Totalresultados: 25
Query:"large language model" AND "medicine" AND "imaging"(25, 12)
Consultando arXiv: "large language model" AND "medicine" AND "decision support"
Obtenidos 21 resultados en este lote (total: 

ERROR: Could not find a version that satisfies the requirement google.colab (from versions: none)
ERROR: No matching distribution found for google.colab


ERROR: Could not find a version that satisfies the requirement google.colab (from versions: none)
ERROR: No matching distribution found for google.colab


In [80]:
print(f"Arvix queries accounts {df_arxiv.shape[0]} rows and {df_arxiv.shape[1]} columns.")

Arvix queries accounts 1613 rows and 12 columns.


In [81]:
df_arxiv.columns

Index(['title', 'summary', 'published', 'updated', 'arxiv_url', 'pdf_url',
       'authors', 'categories', 'doi', 'year', 'primary_category', 'query'],
      dtype='object')

In [82]:
df_arxiv.sample(4)

,title,summary,published,updated,arxiv_url,pdf_url,authors,categories,doi,year,primary_category,query
55,Extrinsically-Focused Evaluation of Omissions ...,Large language models (LLMs) have shown promis...,2023-11-14,2024-11-11,http://arxiv.org/abs/2311.08303v2,http://arxiv.org/pdf/2311.08303v2,"Elliot Schumacher, Daniel Rosenthal, Dhruv Nai...","cs.CL, cs.AI",None,2023,cs.CL,"""LLM"" AND ""healthcare"" AND ""diagnosis"""
7,Fine-Tuned Large Language Models for Symptom R...,The accurate recognition of symptoms in clinic...,2024-01-28,2024-01-28,http://arxiv.org/abs/2401.15780v1,http://arxiv.org/pdf/2401.15780v1,"Mai A. Shaaban, Abbas Akkasi, Adnan Khan, Maji...","cs.CL, cs.LG",http://dx.doi.org/10.5281/zenodo.10104138,2024,cs.CL,"""large language model"" AND ""healthcare"" AND ""d..."
15,ColaCare: Enhancing Electronic Health Record M...,"We introduce ColaCare, a framework that enhanc...",2024-10-03,2025-02-26,http://arxiv.org/abs/2410.02551v2,http://arxiv.org/pdf/2410.02551v2,"Zixiang Wang, Yinghao Zhu, Huiya Zhao, Xiaoche...","cs.LG, cs.AI, cs.CL",http://dx.doi.org/10.1145/3696410.3714877,2024,cs.LG,"""LLM"" AND ""medicine"" AND ""decision support"""
34,Retrospective Comparative Analysis of Prostate...,In-basket message interactions play a crucial ...,2024-09-26,2024-09-26,http://arxiv.org/abs/2409.18290v1,http://arxiv.org/pdf/2409.18290v1,"Yuexing Hao, Jason M. Holmes, Jared Hobson, Al...","cs.AI, cs.CY",None,2024,cs.AI,"""large language model"" AND ""cancer"" AND ""treat..."


#### Pubmed Sweep

In [55]:
df_pubmed=pd.DataFrame()
for query in combined_list:
  df_temp =search_pubmed_and_save_csv(query, year_start , year_end, drive_folder_name="PubMed_Results")# search_arxiv(query, max_results=2000, start=0, sort_by='relevance', date_range=None)
  try:
    print("Query:"+query+str(df_temp.shape))
  except:
    print("Query:"+query+str(None))
  df_pubmed=pd.concat([df_pubmed,df_temp])
  df_pubmed.to_excel("/content/drive/MyDrive/queries_arxiv_v2/queries_pubmed_v2_1.xlsx")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 53 results for query: "large language model" AND "medicine" AND "patient care" AND ("2024/01/01"[PDAT] : "2025/12/31"[PDAT])
Query:"large language model" AND "medicine" AND "patient care"(53, 14)
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 1 results for query: "large language model" AND "medicine" AND "patient monitoring" AND ("2024/01/01"[PDAT] : "2025/12/31"[PDAT])
Query:"large language model" AND "medicine" AND "patient monitoring"(1, 14)
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 124 results for query: "large language model" AND "medicine" AND "imaging" AND ("2024/01/01"[PDAT] : "2025/12/31"[PDAT])
Query:"large language model" AND "medicine" AND "imaging"(124, 14)
Drive

In [48]:
df.head()

,PubMed ID,title,summary,Journal,Publication Date,authors,MeSH Terms,Keywords,Article Type,Volume,Issue,Pages,DOI
0,40502931,Leveraging large language models for accurate ...,,Computational and structural biotechnology jou...,,"Spitzl, Daniel; Mergen, Markus; Bauer, Ulrike;...",,,,27,,2139-2146,
1,40490095,Priming with specific context improves large l...,,Journal of nuclear cardiology : official publi...,,"Hijazi, Waseem; Builoff, Valerie; Killekar, Ad...",,,,,,102269,
2,40470039,Enhancing gastroenterology with multimodal lea...,,Frontiers in medicine,,"Qin, Yuanyuan; Chang, Jianming; Li, Li; Wu, Mi...",,,,12,,1583514,
3,40451964,Fine-tuned large Language model for extracting...,,Emergency radiology,,"Fujita, Nana; Yasaka, Koichiro; Kiryu, Shigeru...",,,,,,,
4,40438192,Performance evaluation of large language model...,,Digital health,,"Kuribara, Tomoki; Hirayama, Kengo; Hirata, Kenji",,,,11,,20552076251346571,


In [72]:
df_arxiv['query'].value_counts().reset_index()

,query,count
0,query,1542


In [67]:
df_pubmed_query_values= df_pubmed['query'].value_counts().reset_index()
df_arxiv_query_values= df_arxiv['query'].value_counts().reset_index()
df_merged_query_values = df_arxiv_query_values.merge(df_pubmed_query_values, on='query', how='outer', suffixes=('_arxiv', '_pubmed'))
df_merged_query_values

,query,count_arxiv,count_pubmed
0,"""LLM"" AND ""cancer"" AND ""decision support""",NaN,19.0
1,"""LLM"" AND ""cancer"" AND ""diagnosis""",NaN,46.0
2,"""LLM"" AND ""cancer"" AND ""hallucination""",NaN,8.0
3,"""LLM"" AND ""cancer"" AND ""imaging""",NaN,42.0
4,"""LLM"" AND ""cancer"" AND ""patient care""",NaN,8.0
5,"""LLM"" AND ""cancer"" AND ""question answering""",NaN,11.0
6,"""LLM"" AND ""cancer"" AND ""treatment""",NaN,66.0
7,"""LLM"" AND ""healthcare"" AND ""decision support""",NaN,45.0
8,"""LLM"" AND ""healthcare"" AND ""diagnosis""",NaN,70.0
9,"""LLM"" AND ""healthcare"" AND ""hallucination""",NaN,9.0


In [56]:
df_pubmed.shape

(2342, 14)

In [62]:
# prompt: devuelveme un excel concatenando df_pubmed y  df_arxiv y eliminar duplicados por titulo manteniendo en columna las queries de las filas duplicadas

# Assuming df_pubmed and df_arxiv DataFrames are already created from the preceding code

# Concatenate the two dataframes
df_combined = pd.concat([df_pubmed, df_arxiv], ignore_index=True)

# Group by 'title' and aggregate the 'query' column
# We'll create a list of unique queries for each title
df_combined['query'] = df_combined.groupby('title')['query'].transform(lambda x: ', '.join(x.dropna().unique()))

# Drop duplicate rows based on the 'title' column, keeping the first occurrence
df_unique = df_combined.drop_duplicates(subset=['title'], keep='first')

# Define the path to save the Excel file in Google Drive
#excel_filepath = '/content/drive/MyDrive/combined_papers_unique.xlsx'

# Save the unique dataframe to an Excel file
#df_unique.to_excel(excel_filepath, index=False)

#print(f"Combined, de-duplicated data saved to: {excel_filepath}")

In [63]:
df_unique.head(5)

,PubMed ID,title,summary,Journal,Publication Date,authors,MeSH Terms,Keywords,Article Type,Volume,Issue,Pages,DOI,query,Unnamed: 0,published,updated,arxiv_url,pdf_url,categories,doi,year,primary_category
0,40505763,RadGPT: A system based on a large language mod...,,Journal of the American College of Radiology :...,,"Herwald, Sanna E; Shah, Preya; Johnston, Andre...",,,,,,,,"""large language model"" AND ""medicine"" AND ""pat...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40491696,Evaluating the Application of Artificial Intel...,,"Clinical ophthalmology (Auckland, N.Z.)",,"Patel, Neeket R; Lacher, Corey R; Huang, Alan ...",,,,19,,1763-1769,,"""large language model"" AND ""medicine"" AND ""pat...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,40435166,"Physician awareness of, interest in, and curre...",,PloS one,,"Solmonovich, Rachel L; Kouba, Insaf; Lee, Ji Y...",,,,20,5,e0320749,,"""large language model"" AND ""medicine"" AND ""pat...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,40423065,The Accuracy of ChatGPT-4o in Interpreting Che...,,Journal of personalized medicine,,"Lacaita, Pietro G; Galijasevic, Malik; Swoboda...",,,,15,5,,,"""large language model"" AND ""medicine"" AND ""pat...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,40378254,Semi-automated pipeline to accelerate multi-si...,,Journal of the American Medical Informatics As...,,"Fan, Hao; Rossetti, Sarah C; Thate, Jennifer; ...",,,,,,,,"""large language model"" AND ""medicine"" AND ""pat...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
pd.set_option('display.max_columns', None)



  PubMed ID                                              title summary  \
0  40505763  RadGPT: A system based on a large language mod...           
1  40491696  Evaluating the Application of Artificial Intel...           
2  40435166  Physician awareness of, interest in, and curre...           
3  40423065  The Accuracy of ChatGPT-4o in Interpreting Che...           
4  40378254  Semi-automated pipeline to accelerate multi-si...           

                                             Journal Publication Date  \
0  Journal of the American College of Radiology :...                    
1            Clinical ophthalmology (Auckland, N.Z.)                    
2                                           PloS one                    
3                   Journal of personalized medicine                    
4  Journal of the American Medical Informatics As...                    

                                             authors MeSH Terms Keywords  \
0  Herwald, Sanna E; Shah, Preya; Johnst